# Etapa 1: Carga y preprocesamiento de los datos

<p align="justify">En esta primera etapa del trabajo se realizarán los procesos relacionados con la carga y el preprocesado de los datos. Dentro de estos trabajos se incluyen la carga, la integración, la selección, la reducción, la conversión y la limpieza de los datos.
<p align="justify">Los datos han sido extraídos directamente del GDC Portal Data y corresponden a los contenidos en la categoría clinical del proyecto TCGA-PAAD. Dentro de esta categoría se encuentran los siguientes archivos:

- clinical.tsv
- exposure.tsv
- family.tsv
- follow_up.tsv
- path_details.csv
<p align="justify">El archivo clinical es el principal y contiene los datos básicos de los pacientes y sus diagnósticos y tratamientos. El archivo exposure contiene información sobre exposición a alcohol y tabaco. El archivo family contiene información sobre el historial familiar de cáncer. El archivo follow_up contiene información sobre el seguimiento de los pacientes. El archivo path_details contiene datos histopatológicos adicionales.
<p align="justify">Todos los conjuntos de datos resultantes contienen el mismo identificador, Patient Id, que permite relacionar la información de cualquiera de ellos entre sí.
<p align="justify">A lo largo de este notebook se irán describiendo los procesos realizados con su pertinente justificación. También se realizará una descripción de las columnas resultantes.






In [3]:
# Montamos la unidad de Google Drive.
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/MyDrive/TFM

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/TFM


In [4]:
# Importamos módulos necesarios.
import requests
import os

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from utils import read_data, save_data

# Aplicamos configuración sobre la visualización de los DataFrames.
pd.set_option('future.no_silent_downcasting', True)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Ruta de los archivos de origen.
input_path = 'data/raw/'
# Ruta de los archivos procesados.
output_path = 'data/processed/'

## 1. Carga y preprocesado inicial

<p align="justify">Los archivos sin procesar están almacenados en la ruta data/raw y los archivos procesados se guardarán en la ruta data/processed. Como se ha comentado anteriormente, los archivos han sido descargados manualmente y almacenados en el directorio indicado.
<p align="justify">La limpieza inicial de cada conjunto de datos es común en todos ellos y se realiza a través de la función replace_data. En ella, primero de todo, se substituyen diferentes formatos para indicar los valores nulos por un formato nulo reconocido. Posteriormente, se eliminan todas las columnas que solo contengan valores nulos. Esto es debido a que todos los proyectos del TCGA comparten las mismas columnas, pero solo algunas de ellas se usan en cada proyecto. Por lo cual, todos los conjuntos de datos contienen gran cantidad de columnas vacías.
<p align="justify">Podemos comprobar la presencia de gran cantidad de columnas vacías comprobando como varia la forma de los diferentes DataFrame tras realizar la limpieza.

In [5]:
def replace_data(df):
    """ Recibe un dataframe y devuelve un dataframe limpio substituyendo ''--'
        por NaN y eliminando columnas con todos los valores NaN.
    """

    df.replace(["'--", 'Unknown', 'unknown', 'not reported', 'Not Reported'],
               np.nan, inplace=True)
    df.dropna(how='all', inplace=True, axis=1)

    return df

In [63]:
# Cargamos todos los archivos.
clinical = read_data(f"{input_path}clinical.tsv", sep="\t")
exposure = read_data(f"{input_path}exposure.tsv", sep="\t")
family = read_data(f"{input_path}family_history.tsv", sep="\t")
follow_up = read_data(f"{input_path}follow_up.tsv", sep="\t")
path_details = read_data(f"{input_path}pathology_detail.tsv", sep="\t")

In [7]:
# Mostramos la forma original de los DataFrame.
clinical.shape, exposure.shape, family.shape, follow_up.shape, path_details.shape

((1082, 210), (290, 40), (169, 13), (687, 198), (185, 86))

In [64]:
# Realizamos la primera limpieza.
clinical = replace_data(clinical)
exposure = replace_data(exposure)
family = replace_data(family)
follow_up = replace_data(follow_up)
path_details = replace_data(path_details)

In [9]:
# Verificamos la reducción del número de columnas.
clinical.shape, exposure.shape, family.shape, follow_up.shape, path_details.shape

((1082, 66), (290, 12), (169, 5), (687, 22), (185, 11))

## 2. Preprocesado de follow_up.tsv

<p align="justify">Primero de todo se realiza una exploración de los datos para familiarizarse con ellos y detectar aquellas columnas que contienen información de interés.
<p align="justify">Follow up contiene los datos sobre los seguimientos a los pacientes. Tras la exploración se observa que las únicas filas de interés son aquellas que pertenecen a la categoría temporal 'último contacto' o 'previo al diagnóstico'. Se filtrarán esas filas y se dividiran los datos según esas dos categorías.

In [10]:
# Mostramos los datos.
print(follow_up.shape)
follow_up.head()

(687, 22)


,project.project_id,cases.case_id,cases.submitter_id,follow_ups.days_to_follow_up,follow_ups.days_to_progression,follow_ups.days_to_recurrence,follow_ups.disease_response,follow_ups.evidence_of_progression_type,follow_ups.evidence_of_recurrence_type,follow_ups.follow_up_id,follow_ups.progression_or_recurrence,follow_ups.progression_or_recurrence_anatomic_site,follow_ups.progression_or_recurrence_type,follow_ups.submitter_id,follow_ups.timepoint_category,other_clinical_attributes.comorbidities,other_clinical_attributes.days_to_comorbidity,other_clinical_attributes.days_to_risk_factor,other_clinical_attributes.other_clinical_attribute_id,other_clinical_attributes.risk_factors,other_clinical_attributes.submitter_id,other_clinical_attributes.timepoint_category
0,TCGA-PAAD,01775b06-5836-469c-8537-120cb8cc94e9,TCGA-IB-7897,486,NaN,NaN,WT-With Tumor,NaN,NaN,1044032b-d9c5-44e2-8b69-03684f9ae4ff,NaN,NaN,NaN,TCGA-IB-7897_follow_up2,Follow-up,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,TCGA-PAAD,01775b06-5836-469c-8537-120cb8cc94e9,TCGA-IB-7897,486,NaN,NaN,WT-With Tumor,NaN,NaN,2e90d560-ea0d-4373-b7c6-e724d3786bff,NaN,NaN,NaN,TCGA-IB-7897_follow_up,Last Contact,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,TCGA-PAAD,01775b06-5836-469c-8537-120cb8cc94e9,TCGA-IB-7897,397,NaN,397,NaN,NaN,Convincing Image Source,5226aa9a-6783-4b4f-86c4-e74289c37f59,Yes,Liver,Locoregional,TCGA-IB-7897_follow_up3,Post Initial Treatment,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,TCGA-PAAD,01775b06-5836-469c-8537-120cb8cc94e9,TCGA-IB-7897,397,397,NaN,NaN,Convincing Image Source,NaN,63115a69-8a13-4280-b308-c725936e6593,Yes,Liver,Distant,TCGA-IB-7897_follow_up4,Post Initial Treatment,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,TCGA-PAAD,02dbd5fa-e31f-4486-8df8-5b851f2e92bd,TCGA-HZ-8315,28,NaN,NaN,WT-With Tumor,NaN,NaN,29bdf41f-cfb1-40eb-a151-c8175fbde0ec,NaN,NaN,NaN,TCGA-HZ-8315_follow_up2,Follow-up,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
# Realizamos una exploración detallada de la información
# contenida en cada columna.
for col in follow_up.columns:
    print(f"{col}: {follow_up[col].unique()}")

project.project_id: ['TCGA-PAAD']
cases.case_id: ['01775b06-5836-469c-8537-120cb8cc94e9'
 '02dbd5fa-e31f-4486-8df8-5b851f2e92bd'
 '02ef2e1a-5afe-4c00-a3ea-7f4a153f1da4'
 '05d2adb3-5c6d-4edc-ba4c-bdbf628f4eee'
 '0944ce65-a89a-4916-b90a-f674b334281e'
 '09fd6d9e-c715-4d57-88f9-238dd74c79a7'
 '0a3c8161-4186-4c95-8288-9928d6db0355'
 '0b4443a0-087d-47e7-9e67-795eac6750c3'
 '0b4b50c7-aa0b-4a64-807b-d3cb47eeeb6f'
 '0fa89fbc-307a-48d5-adce-192556755c22'
 '10f0d40b-c4c4-441b-9cfb-bef5e44aa91e'
 '13cf4e4c-8d5f-4854-a350-428fdf1ff340'
 '1422bc35-4597-4e76-aa75-8e79613e67d5'
 '1521ac6d-31f7-4240-9d54-4f7600e78e16'
 '16b38977-aea1-4c75-89ec-4fb551f652dd'
 '17142718-742e-464a-8e0f-a6b9ac8fad75'
 '1756244e-13bc-4ed7-8b91-005608803d91'
 '1782bf45-7350-4053-8dd1-45faf4ed8dd9'
 '178e68c0-4cb8-4042-a942-5a159ec1e12b'
 '18960eb6-2d79-4d12-886a-4cd1e8fc1e21'
 '1b31d2e9-5b05-4fff-88d6-3a95bf8f8d43'
 '1c2341f3-558b-4555-90ff-c7a23ef85c0b'
 '1db061c0-9b28-4bf9-8ad2-0383c69b6800'
 '1f5494c4-e26c-4512-96d9-30412

In [12]:
# Elimnación columnas innecesarias.
follow_up.drop(['project.project_id', 'cases.case_id',
                'follow_ups.submitter_id'], axis=1, inplace=True)

# Formateo nombres columnas.
follow_up.columns = follow_up.columns.str.replace('cases.', '')\
      .str.replace('follow_ups.', '').str.replace('submitter', 'patient')\
      .str.replace('_', ' ').str.title()

### 2.1 Preprocesado de follow_up_last

<p align="justify">Se seleccionan aquellas filas donde el seguimiento ha sido clasificado como último contacto. Se eliminan todas las columnas que no contienen información de interés. La información resultante se incorporará al DataFrame demographic. Las columnas resultantes son:

- Patient Id: Identificador.
- Days To Follow Up: Días transcurridos hasta el último seguimiento.

In [13]:
# Filtrado de filas.
follow_up_last = follow_up[follow_up['Timepoint Category'] == 'Last Contact'].copy()

# Selección de columnas.
follow_up_last.reset_index(drop=True, inplace=True)
follow_up_last = follow_up_last[['Patient Id', 'Days To Follow Up']]

print(follow_up_last.shape)
follow_up_last

(185, 2)


,Patient Id,Days To Follow Up
0,TCGA-IB-7897,486
1,TCGA-HZ-8315,299
2,TCGA-HZ-8636,545
3,TCGA-IB-7888,1332
4,TCGA-IB-7893,117
5,TCGA-US-A774,695
6,TCGA-IB-AAUP,431
7,TCGA-IB-8127,522
8,TCGA-2J-AABU,277
9,TCGA-HV-AA8V,920


### 2.2 Preprocesado de comorbidities

<p align="justify">Se seleccionan aquellas filas donde el seguimiento ha sido clasificado como previo al diagnóstico. Se eliminan todas las columnas que no contienen información de interés. La información resultante se incorporará al DataFrame demographic. Las columnas resultantes son:

- Patient Id: Identificador.
- Comorbidities: Comorbilidad que tiene el paciente.

In [14]:
# Filtrado filas.
commorbidities = follow_up[follow_up['Other Clinical Attributes.Timepoint Category'] == 'Prior to Diagnosis'].copy()

# Selección columnas.
commorbidities = commorbidities[['Patient Id', 'Other Clinical Attributes.Comorbidities']]

# Nombre columnas.
commorbidities.columns = ['Patient Id', 'Comorbidities']
commorbidities.reset_index(drop=True, inplace=True)

print(commorbidities.shape)
commorbidities

(51, 2)


,Patient Id,Comorbidities
0,TCGA-IB-7893,Diabetes
1,TCGA-IB-AAUP,Diabetes
2,TCGA-3A-A9IH,Chronic Pancreatitis
3,TCGA-3A-A9IO,Diabetes
4,TCGA-Q3-A5QY,Diabetes
5,TCGA-IB-7644,Diabetes
6,TCGA-IB-AAUW,Diabetes
7,TCGA-F2-A7TX,Diabetes
8,TCGA-2J-AABI,Chronic Pancreatitis
9,TCGA-HV-A7OL,Diabetes


## 3. Preprocesado de exposure.tsv

<p align="justify">Primero de todo se realiza una exploración de los datos para familiarizarse con ellos y detectar aquellas columnas que contienen información de interés. Exposure contiene información sobre la exposición a alcohol o tábaco por parte de los pacientes.

<p align="justify">Se eliminan todas aquellas columnas sin información de interés y se adecuan los nombres de las columnas. Se agrupan las filas por Patient Id, ya que, inicialmente se utilizan dos filas por paciente, una para el alcohol y la otra para el tábaco. Finalmente, se eliminan las filas vacías y se seleccionan las columnas de interés. La información resultante se incorporará a demographic. Las columnas resultantes son:

- Patient Id: Identificador.
- Alcohol History: Presencia de historial de alcohol.
- Alcohol Intensity: Tipo de consumidor alcohol.
- Tobacco Smoking Status: Tipo de fumador.

In [15]:
# Mostramos los datos.
print(exposure.shape)
exposure.head()

(290, 12)


,project.project_id,cases.case_id,cases.submitter_id,exposures.alcohol_days_per_week,exposures.alcohol_drinks_per_day,exposures.alcohol_history,exposures.alcohol_intensity,exposures.exposure_type,exposures.pack_years_smoked,exposures.tobacco_smoking_onset_year,exposures.tobacco_smoking_quit_year,exposures.tobacco_smoking_status
0,TCGA-PAAD,01775b06-5836-469c-8537-120cb8cc94e9,TCGA-IB-7897,NaN,NaN,Yes,Social Drinker,NaN,NaN,NaN,NaN,NaN
1,TCGA-PAAD,01775b06-5836-469c-8537-120cb8cc94e9,TCGA-IB-7897,NaN,NaN,NaN,NaN,Tobacco,NaN,NaN,1981,"Current Reformed Smoker, Duration Not Specified"
2,TCGA-PAAD,02dbd5fa-e31f-4486-8df8-5b851f2e92bd,TCGA-HZ-8315,NaN,NaN,NaN,NaN,Tobacco,NaN,NaN,NaN,NaN
3,TCGA-PAAD,02ef2e1a-5afe-4c00-a3ea-7f4a153f1da4,TCGA-HZ-8636,NaN,NaN,NaN,NaN,Tobacco,NaN,NaN,NaN,NaN
4,TCGA-PAAD,05d2adb3-5c6d-4edc-ba4c-bdbf628f4eee,TCGA-IB-7888,NaN,NaN,NaN,NaN,Tobacco,NaN,NaN,NaN,NaN


In [16]:
# Realizamos una exploración detallada de la información
# contenida en cada columna.
for col in exposure.columns:
    print(f"{col}: {exposure[col].unique()}")

project.project_id: ['TCGA-PAAD']
cases.case_id: ['01775b06-5836-469c-8537-120cb8cc94e9'
 '02dbd5fa-e31f-4486-8df8-5b851f2e92bd'
 '02ef2e1a-5afe-4c00-a3ea-7f4a153f1da4'
 '05d2adb3-5c6d-4edc-ba4c-bdbf628f4eee'
 '0944ce65-a89a-4916-b90a-f674b334281e'
 '09fd6d9e-c715-4d57-88f9-238dd74c79a7'
 '0a3c8161-4186-4c95-8288-9928d6db0355'
 '0b4443a0-087d-47e7-9e67-795eac6750c3'
 '0b4b50c7-aa0b-4a64-807b-d3cb47eeeb6f'
 '0fa89fbc-307a-48d5-adce-192556755c22'
 '10f0d40b-c4c4-441b-9cfb-bef5e44aa91e'
 '13cf4e4c-8d5f-4854-a350-428fdf1ff340'
 '1422bc35-4597-4e76-aa75-8e79613e67d5'
 '1521ac6d-31f7-4240-9d54-4f7600e78e16'
 '16b38977-aea1-4c75-89ec-4fb551f652dd'
 '17142718-742e-464a-8e0f-a6b9ac8fad75'
 '1756244e-13bc-4ed7-8b91-005608803d91'
 '1782bf45-7350-4053-8dd1-45faf4ed8dd9'
 '178e68c0-4cb8-4042-a942-5a159ec1e12b'
 '18960eb6-2d79-4d12-886a-4cd1e8fc1e21'
 '1b31d2e9-5b05-4fff-88d6-3a95bf8f8d43'
 '1c2341f3-558b-4555-90ff-c7a23ef85c0b'
 '1db061c0-9b28-4bf9-8ad2-0383c69b6800'
 '1f5494c4-e26c-4512-96d9-30412

In [17]:
# Formateo nombres columnas.
exposure.columns = exposure.columns.str.replace('cases.', '')\
      .str.replace('exposures.', '').str.replace('submitter', 'patient')\
      .str.replace('_', ' ').str.title()

# Filtrado columnas de interés.
exposure = exposure[['Patient Id', 'Alcohol History',
                     'Alcohol Intensity', 'Tobacco Smoking Status']].copy()

# Agrupación por Patient Id.
exposure = exposure.groupby('Patient Id', as_index=False).sum()

# Eliminación filas sin información.
exposure.replace(0, np.nan, inplace=True)
exposure.dropna(thresh=2, inplace=True)
exposure.reset_index(drop=True, inplace=True)

print(exposure.shape)
exposure

(150, 4)


,Patient Id,Alcohol History,Alcohol Intensity,Tobacco Smoking Status
0,TCGA-2J-AAB1,Yes,Drinker,Current Reformed Smoker for > 15 yrs
1,TCGA-2J-AAB4,NaN,NaN,"Current Reformed Smoker, Duration Not Specified"
2,TCGA-2J-AAB9,Yes,Drinker,Lifelong Non-Smoker
3,TCGA-2J-AABA,NaN,NaN,"Current Reformed Smoker, Duration Not Specified"
4,TCGA-2J-AABE,Yes,Drinker,Current Reformed Smoker for > 15 yrs
5,TCGA-2J-AABF,Yes,Drinker,Current Reformed Smoker for > 15 yrs
6,TCGA-2J-AABH,NaN,NaN,Lifelong Non-Smoker
7,TCGA-2J-AABI,Yes,Social Drinker,Current Reformed Smoker for < or = 15 yrs
8,TCGA-2J-AABO,NaN,NaN,Current Reformed Smoker for < or = 15 yrs
9,TCGA-2J-AABP,Yes,Drinker,Lifelong Non-Smoker


## 4. Preprocesado de family.tsv

<p align="justify">Primero de todo se realiza una exploración de los datos para familiarizarse con ellos y detectar aquellas columnas que contienen información de interés. Family contiene información sobre el historial familiar de cáncer.

<p align="justify">Se eliminan todas aquellas columnas sin información de interés y se adecuan los nombres de las columnas. Se eliminan las filas vacías y las entradas duplicadas. Hay algunos pacientes que tienen dos entradas, una con cáncer y otra especificando el tipo de cáncer. La información resultante se incorporará a demographic. Las columnas resultantes son:

- Patient Id: Identificador.
- Relative With Cancer: Si existen familiares con historial de cáncer.
- Relative Cancer Type: Tipo de cáncer del familiar.

In [18]:
# Mostramos los datos.
print(family.shape)
family.head()

(169, 5)


,project.project_id,cases.case_id,cases.submitter_id,family_histories.relationship_primary_diagnosis,family_histories.relative_with_cancer_history
0,TCGA-PAAD,01775b06-5836-469c-8537-120cb8cc94e9,TCGA-IB-7897,Melanoma,yes
1,TCGA-PAAD,01775b06-5836-469c-8537-120cb8cc94e9,TCGA-IB-7897,Cancer,yes
2,TCGA-PAAD,02dbd5fa-e31f-4486-8df8-5b851f2e92bd,TCGA-HZ-8315,NaN,NaN
3,TCGA-PAAD,02ef2e1a-5afe-4c00-a3ea-7f4a153f1da4,TCGA-HZ-8636,NaN,NaN
4,TCGA-PAAD,05d2adb3-5c6d-4edc-ba4c-bdbf628f4eee,TCGA-IB-7888,NaN,NaN


In [19]:
# Realizamos una exploración detallada de la información
# contenida en cada columna.
for col in family.columns:
    print(f"{col}: {family[col].unique()}")

project.project_id: ['TCGA-PAAD']
cases.case_id: ['01775b06-5836-469c-8537-120cb8cc94e9'
 '02dbd5fa-e31f-4486-8df8-5b851f2e92bd'
 '02ef2e1a-5afe-4c00-a3ea-7f4a153f1da4'
 '05d2adb3-5c6d-4edc-ba4c-bdbf628f4eee'
 '0944ce65-a89a-4916-b90a-f674b334281e'
 '09fd6d9e-c715-4d57-88f9-238dd74c79a7'
 '0a3c8161-4186-4c95-8288-9928d6db0355'
 '0b4443a0-087d-47e7-9e67-795eac6750c3'
 '0b4b50c7-aa0b-4a64-807b-d3cb47eeeb6f'
 '10f0d40b-c4c4-441b-9cfb-bef5e44aa91e'
 '13cf4e4c-8d5f-4854-a350-428fdf1ff340'
 '1422bc35-4597-4e76-aa75-8e79613e67d5'
 '1521ac6d-31f7-4240-9d54-4f7600e78e16'
 '17142718-742e-464a-8e0f-a6b9ac8fad75'
 '1756244e-13bc-4ed7-8b91-005608803d91'
 '1782bf45-7350-4053-8dd1-45faf4ed8dd9'
 '178e68c0-4cb8-4042-a942-5a159ec1e12b'
 '18960eb6-2d79-4d12-886a-4cd1e8fc1e21'
 '1c2341f3-558b-4555-90ff-c7a23ef85c0b'
 '1db061c0-9b28-4bf9-8ad2-0383c69b6800'
 '1f5494c4-e26c-4512-96d9-30412b5b32ce'
 '1fe937f7-b4e5-4cf6-be9b-215a012533c6'
 '20094e3e-cee3-41dd-8553-1fc393117786'
 '2048d0f9-ed9f-46df-bcd4-3dfd9

In [20]:
# Formateo columnas.
family.columns = family.columns.str.replace('cases.', '')\
      .str.replace('family_histories.', '').str.replace('submitter', 'patient')\
      .str.replace('_', ' ').str.title()

# Eliminación columnas innecesarias.
family.drop(columns=['Project.Project Id', 'Case Id'], inplace=True)

# Eliminación filas vacías y duplicadas.
family.dropna(thresh=2, inplace=True)
family = family.groupby(['Patient Id', 'Relative With Cancer History'], as_index=False).max()
family.replace(0, np.nan, inplace=True)

# Nombres columnas.
family.columns = ['Patient Id', 'Relative With Cancer', 'Relative Cancer Type']

print(family.shape)
family

(113, 3)


,Patient Id,Relative With Cancer,Relative Cancer Type
0,TCGA-2J-AAB1,no,NaN
1,TCGA-2J-AAB9,yes,Pancreas Cancer
2,TCGA-2J-AABE,no,NaN
3,TCGA-2J-AABF,yes,Cancer
4,TCGA-2J-AABH,yes,Cancer
5,TCGA-2J-AABI,no,NaN
6,TCGA-2J-AABO,yes,Cancer
7,TCGA-2J-AABP,yes,Cancer
8,TCGA-2J-AABT,no,NaN
9,TCGA-2J-AABU,no,NaN


## 5. Preprocesado de path_details.tsv

<p align="justify">Primero de todo se realiza una exploración de los datos para familiarizarse con ellos y detectar aquellas columnas que contienen información de interes. Path_details contiene información sobre detalles relacionados con el tumor y su diagnóstico.

<p align="justify">Se filtran las columnas de interés, se adecua el nombre de las columnas y se eliminan aquellas filas que estan vacías. La información resultante se incorporará a diagnosis. Las columnas resultantes son:

- Patient Id: Identificador.
- Lymph Nodes Positive: Número de nódulos limfáticos afectador por el tumor.
- Maximum Tumor Diameter: Diámetro máximo del tumor en centímetros.

In [21]:
# Mostramos los datos.
print(path_details.shape)
path_details.head()

(185, 11)


,project.project_id,cases.case_id,cases.submitter_id,diagnoses.diagnosis_id,diagnoses.submitter_id,pathology_details.consistent_pathology_review,pathology_details.lymph_nodes_positive,pathology_details.lymph_nodes_tested,pathology_details.pathology_detail_id,pathology_details.submitter_id,pathology_details.tumor_largest_dimension_diameter
0,TCGA-PAAD,01775b06-5836-469c-8537-120cb8cc94e9,TCGA-IB-7897,12c51627-192a-5f84-b7e5-d05c096f6536,TCGA-IB-7897_diagnosis,Yes,2,17,c5b9fa0d-5933-4d94-9ba5-43c2af8f58ac,TCGA-IB-7897_pathology_detail,3.5
1,TCGA-PAAD,02dbd5fa-e31f-4486-8df8-5b851f2e92bd,TCGA-HZ-8315,c82a396e-5acf-56c0-a14f-40a139a8f528,TCGA-HZ-8315_diagnosis,Yes,0,21,5eab9525-ed78-4b03-a2a8-d7b03a07cd43,TCGA-HZ-8315_pathology_detail,6.0
2,TCGA-PAAD,02ef2e1a-5afe-4c00-a3ea-7f4a153f1da4,TCGA-HZ-8636,6dda35c2-bcd9-58a7-8589-b0f2a0da5409,TCGA-HZ-8636_diagnosis,Yes,0,2,d719d190-756d-4275-8c9f-d91ac241a989,TCGA-HZ-8636_pathology_detail,9.0
3,TCGA-PAAD,05d2adb3-5c6d-4edc-ba4c-bdbf628f4eee,TCGA-IB-7888,7f364e92-505d-569f-bf26-b620a41a406e,TCGA-IB-7888_diagnosis,Yes,0,12,8297c80a-f0c2-4a56-ac9d-1b07aed20886,TCGA-IB-7888_pathology_detail,3.0
4,TCGA-PAAD,0944ce65-a89a-4916-b90a-f674b334281e,TCGA-IB-7893,21f56cab-cdd4-5a50-9a71-a014bd2b88b4,TCGA-IB-7893_diagnosis,Yes,0,8,a19eff5e-266f-44d0-aef9-b5e8e9cdfb64,TCGA-IB-7893_pathology_detail,7.0


In [22]:
# Realizamos una exploración detallada de la información
# contenida en cada columna.
for col in path_details.columns:
    print(f"{col}: {path_details[col].unique()}")

project.project_id: ['TCGA-PAAD']
cases.case_id: ['01775b06-5836-469c-8537-120cb8cc94e9'
 '02dbd5fa-e31f-4486-8df8-5b851f2e92bd'
 '02ef2e1a-5afe-4c00-a3ea-7f4a153f1da4'
 '05d2adb3-5c6d-4edc-ba4c-bdbf628f4eee'
 '0944ce65-a89a-4916-b90a-f674b334281e'
 '09fd6d9e-c715-4d57-88f9-238dd74c79a7'
 '0a3c8161-4186-4c95-8288-9928d6db0355'
 '0b4443a0-087d-47e7-9e67-795eac6750c3'
 '0b4b50c7-aa0b-4a64-807b-d3cb47eeeb6f'
 '0fa89fbc-307a-48d5-adce-192556755c22'
 '10f0d40b-c4c4-441b-9cfb-bef5e44aa91e'
 '13cf4e4c-8d5f-4854-a350-428fdf1ff340'
 '1422bc35-4597-4e76-aa75-8e79613e67d5'
 '1521ac6d-31f7-4240-9d54-4f7600e78e16'
 '16b38977-aea1-4c75-89ec-4fb551f652dd'
 '17142718-742e-464a-8e0f-a6b9ac8fad75'
 '1756244e-13bc-4ed7-8b91-005608803d91'
 '1782bf45-7350-4053-8dd1-45faf4ed8dd9'
 '178e68c0-4cb8-4042-a942-5a159ec1e12b'
 '18960eb6-2d79-4d12-886a-4cd1e8fc1e21'
 '1b31d2e9-5b05-4fff-88d6-3a95bf8f8d43'
 '1c2341f3-558b-4555-90ff-c7a23ef85c0b'
 '1db061c0-9b28-4bf9-8ad2-0383c69b6800'
 '1f5494c4-e26c-4512-96d9-30412

In [23]:
# Selección columnas.
path_details = path_details[['cases.submitter_id',
                             'pathology_details.lymph_nodes_positive',
                             'pathology_details.tumor_largest_dimension_diameter']]

# Formateo columnas.
path_details.columns = ['Patient Id', 'Lymph Nodes Positive',
                        'Maximum Tumor Diameter (cm)']

# Eliminación filas vacías.
path_details.dropna(thresh=2, inplace=True)

print(path_details.shape)
path_details

(184, 3)


,Patient Id,Lymph Nodes Positive,Maximum Tumor Diameter (cm)
0,TCGA-IB-7897,2,3.5
1,TCGA-HZ-8315,0,6.0
2,TCGA-HZ-8636,0,9.0
3,TCGA-IB-7888,0,3.0
4,TCGA-IB-7893,0,7.0
5,TCGA-US-A774,2,4.0
6,TCGA-IB-AAUP,7,2.7
7,TCGA-IB-8127,3,4.5
8,TCGA-2J-AABU,2,3.3
9,TCGA-HV-AA8V,3,NaN


## 6. Preprocesado de clinicals.tsv

<p align="justify">Primero de todo se realiza una exploración de los datos para familiarizarse con ellos y detectar aquellas columnas que contienen información de interés. Dado el tamaño de este conjunto de datos se decide dividirlo en tres partes para poder ir transformando los datos de forma adecuada y extraer el máximo de información posible.
<p align="justify">Demographic contiene los datos básicos de los pacientes. Diagnosis contiene información sobre los diferentes diagnósticos de los pacientes. Treatment contiene información sobre los diferentes tratamientos que reciben los pacientes.

In [24]:
# Mostramos los datos.
print(clinical.shape)
clinical.head()

(1082, 66)


,project.project_id,cases.case_id,cases.consent_type,cases.days_to_consent,cases.disease_type,cases.index_date,cases.lost_to_followup,cases.primary_site,cases.submitter_id,demographic.age_at_index,demographic.age_is_obfuscated,demographic.cause_of_death,demographic.cause_of_death_source,demographic.country_of_residence_at_enrollment,demographic.days_to_birth,demographic.days_to_death,demographic.demographic_id,demographic.ethnicity,demographic.gender,demographic.race,demographic.submitter_id,demographic.vital_status,diagnoses.age_at_diagnosis,diagnoses.ajcc_pathologic_m,diagnoses.ajcc_pathologic_n,diagnoses.ajcc_pathologic_stage,diagnoses.ajcc_pathologic_t,diagnoses.ajcc_staging_system_edition,diagnoses.classification_of_tumor,diagnoses.days_to_diagnosis,diagnoses.days_to_last_follow_up,diagnoses.diagnosis_id,diagnoses.diagnosis_is_primary_disease,diagnoses.icd_10_code,diagnoses.laterality,diagnoses.method_of_diagnosis,diagnoses.morphology,diagnoses.primary_diagnosis,diagnoses.prior_malignancy,diagnoses.prior_treatment,diagnoses.residual_disease,diagnoses.site_of_resection_or_biopsy,diagnoses.sites_of_involvement,diagnoses.submitter_id,diagnoses.synchronous_malignancy,diagnoses.tissue_or_organ_of_origin,diagnoses.tumor_grade,diagnoses.tumor_grade_category,diagnoses.tumor_of_origin,diagnoses.year_of_diagnosis,treatments.clinical_trial_indicator,treatments.days_to_treatment_end,treatments.days_to_treatment_start,treatments.initial_disease_status,treatments.number_of_fractions,treatments.residual_disease,treatments.submitter_id,treatments.therapeutic_agents,treatments.treatment_anatomic_sites,treatments.treatment_dose,treatments.treatment_dose_units,treatments.treatment_id,treatments.treatment_intent_type,treatments.treatment_or_therapy,treatments.treatment_outcome,treatments.treatment_type
0,TCGA-PAAD,01775b06-5836-469c-8537-120cb8cc94e9,Informed Consent,0,Ductal and Lobular Neoplasms,Diagnosis,No,Pancreas,TCGA-IB-7897,53,False,Cancer Related,Death Certificate,Canada,-19718,486,31f43ced-83c3-5949-9d68-7306832ccc7b,not hispanic or latino,female,white,TCGA-IB-7897_demographic,Dead,19718,M0,N1,Stage IIB,T3,6th,primary,0,486.0,12c51627-192a-5f84-b7e5-d05c096f6536,True,C25.9,NaN,Surgical Resection,8500/3,"Infiltrating duct carcinoma, NOS",no,No,R0,"Pancreas, NOS",Pancreas Head,TCGA-IB-7897_diagnosis,No,"Pancreas, NOS",G2,NaN,NaN,2008,NaN,NaN,NaN,NaN,NaN,NaN,TCGA-IB-7897_treatment2,NaN,NaN,NaN,NaN,11f8ab35-d43b-4a86-adaf-31a177ed4863,Adjuvant,no,NaN,"Radiation Therapy, NOS"
1,TCGA-PAAD,01775b06-5836-469c-8537-120cb8cc94e9,Informed Consent,0,Ductal and Lobular Neoplasms,Diagnosis,No,Pancreas,TCGA-IB-7897,53,False,Cancer Related,Death Certificate,Canada,-19718,486,31f43ced-83c3-5949-9d68-7306832ccc7b,not hispanic or latino,female,white,TCGA-IB-7897_demographic,Dead,19718,M0,N1,Stage IIB,T3,6th,primary,0,486.0,12c51627-192a-5f84-b7e5-d05c096f6536,True,C25.9,NaN,Surgical Resection,8500/3,"Infiltrating duct carcinoma, NOS",no,No,R0,"Pancreas, NOS",Pancreas Head,TCGA-IB-7897_diagnosis,No,"Pancreas, NOS",G2,NaN,NaN,2008,NaN,NaN,NaN,NaN,NaN,NaN,TCGA-IB-7897_treatment,NaN,NaN,NaN,NaN,747e7279-7271-5c39-a57a-5df3bc6145d4,Adjuvant,no,NaN,"Pharmaceutical Therapy, NOS"
2,TCGA-PAAD,01775b06-5836-469c-8537-120cb8cc94e9,Informed Consent,0,Ductal and Lobular Neoplasms,Diagnosis,No,Pancreas,TCGA-IB-7897,53,False,Cancer Related,Death Certificate,Canada,-19718,486,31f43ced-83c3-5949-9d68-7306832ccc7b,not hispanic or latino,female,white,TCGA-IB-7897_demographic,Dead,19718,M0,N1,Stage IIB,T3,6th,primary,0,486.0,12c51627-192a-5f84-b7e5-d05c096f6536,True,C25.9,NaN,Surgical Resection,8500/3,"Infiltrating duct carcinoma, NOS",no,No,R0,"Pancreas, NOS",Pancreas Head,TCGA-IB-7897_diagnosis,No,"Pancreas, NOS",G2,NaN,NaN,2008,NaN,NaN,NaN,NaN,NaN,NaN,TCGA-IB-7897_treatment3,NaN,NaN,NaN,NaN,e24e0cfd-21f6-4b4d-8c08-806806910dc9,NaN,yes,NaN,Whipple
3,TCGA-PAAD,01775b06-5836-469c-8537-120cb8cc94e9,Informed Consent,0,Ductal and Lobular Neoplasms,Diagnosis,No,Pancreas,TCG

In [25]:
# Realizamos una exploración detallada de la información
# contenida en cada columna.
for col in clinical.columns:
    print(f"{col}: {clinical[col].unique()}")

project.project_id: ['TCGA-PAAD']
cases.case_id: ['01775b06-5836-469c-8537-120cb8cc94e9'
 '02dbd5fa-e31f-4486-8df8-5b851f2e92bd'
 '02ef2e1a-5afe-4c00-a3ea-7f4a153f1da4'
 '05d2adb3-5c6d-4edc-ba4c-bdbf628f4eee'
 '0944ce65-a89a-4916-b90a-f674b334281e'
 '09fd6d9e-c715-4d57-88f9-238dd74c79a7'
 '0a3c8161-4186-4c95-8288-9928d6db0355'
 '0b4443a0-087d-47e7-9e67-795eac6750c3'
 '0b4b50c7-aa0b-4a64-807b-d3cb47eeeb6f'
 '0fa89fbc-307a-48d5-adce-192556755c22'
 '10f0d40b-c4c4-441b-9cfb-bef5e44aa91e'
 '13cf4e4c-8d5f-4854-a350-428fdf1ff340'
 '1422bc35-4597-4e76-aa75-8e79613e67d5'
 '1521ac6d-31f7-4240-9d54-4f7600e78e16'
 '16b38977-aea1-4c75-89ec-4fb551f652dd'
 '17142718-742e-464a-8e0f-a6b9ac8fad75'
 '1756244e-13bc-4ed7-8b91-005608803d91'
 '1782bf45-7350-4053-8dd1-45faf4ed8dd9'
 '178e68c0-4cb8-4042-a942-5a159ec1e12b'
 '18960eb6-2d79-4d12-886a-4cd1e8fc1e21'
 '1b31d2e9-5b05-4fff-88d6-3a95bf8f8d43'
 '1c2341f3-558b-4555-90ff-c7a23ef85c0b'
 '1db061c0-9b28-4bf9-8ad2-0383c69b6800'
 '1f5494c4-e26c-4512-96d9-30412

### 6.1 Preprocesado de demographic

<p align="justify">En primer lugar, se analizan los datos demográficos. Se realiza una selección de las columnas con información relevante y se eliminan aquellas filas que están duplicadas. Se uniformiza los nombres de las columnas y algunas categorías. Se incorpora información de otros DataFrame y se guarda el resultado. Las columnas resultantes son:

- Patient Id: Identificador de cada paciente común en todas las tablas.
- Age: Edad inicial del paciente.
- Country Of Residence: País de residencia.
- Ethnicity: Etnia del paciente.
- Gender: Género del paciente.
- Race: Raza del paciente.
- Diabetes: Presencia de diabetes.
- Chronic Pancreatitis: Presencia de pancreatitis crónica.
- Alcohol History: Presencia de historial de alcohol.
- Alcohol Intensity: Tipo de consumidor alcohol.
- Tobacco Smoking Status: Tipo de fumador.
- Relative With Cancer: Si existen familiares con historial de cáncer.
- Relative Cancer Type: Tipo de cáncer del familiar.
- Vital Status: Estado vital del paciente en el último seguimiento.
- Overall Survival: Supervivencia del paciente.

In [26]:
# Selección de columnas.
demographic = clinical[['cases.submitter_id', 'demographic.age_at_index',
                         'demographic.country_of_residence_at_enrollment',
                         'demographic.ethnicity', 'demographic.gender',
                         'demographic.race', 'demographic.days_to_death',
                         ]].copy()

# Eliminación duplicatos.
demographic.drop_duplicates(inplace=True)
demographic.reset_index(drop=True, inplace=True)

# Formateo nombre columnas y categorías.
demographic.columns = demographic.columns.str.replace('cases.', '')\
          .str.replace('demographic.', '').str.replace('submitter', 'patient')\
          .str.replace('_', ' ').str.title().str.replace(' At Enrollment', '')\
          .str.replace(' At Index', '')

# Creación nuevas columnas de commorbidities.
diabetes_id = commorbidities[commorbidities['Comorbidities'] == 'Diabetes']['Patient Id']
pancreatitis_id = commorbidities[commorbidities['Comorbidities'] == 'Chronic Pancreatitis']['Patient Id']

demographic['Diabetes'] = np.where(demographic['Patient Id'].isin(diabetes_id), 'Yes', 'No')
demographic['Chronic Pancreatitis'] = np.where(demographic['Patient Id'].isin(pancreatitis_id), 'Yes', 'No')

# Adición de columnas de exposure.
demographic = demographic.merge(exposure, on='Patient Id', how='left')

# Adición de columnas de family.
demographic = demographic.merge(family, on='Patient Id', how='left')

# Creación de nuevas columnas de follow_up_last.
demographic = demographic.merge(follow_up_last, on='Patient Id', how='left')

demographic['Vital Status'] = clinical['demographic.vital_status'].copy()
demographic['Vital Status'] = demographic['Vital Status'].map({'Alive': 0, 'Dead': 1})

demographic['Days To Death'] = demographic['Days To Death'].astype(float)
demographic['Days To Follow Up'] = demographic['Days To Follow Up'].astype(float)
demographic['Overall Survival'] = demographic[['Days To Death', 'Days To Follow Up']]\
        .max(axis=1).astype(int)

demographic.drop(columns=['Days To Death', 'Days To Follow Up'], inplace=True)

# Guardar datos.
save_data(demographic, f"{output_path}demographic.csv")

print(demographic.shape)
demographic

(185, 15)


,Patient Id,Age,Country Of Residence,Ethnicity,Gender,Race,Diabetes,Chronic Pancreatitis,Alcohol History,Alcohol Intensity,Tobacco Smoking Status,Relative With Cancer,Relative Cancer Type,Vital Status,Overall Survival
0,TCGA-IB-7897,53,Canada,not hispanic or latino,female,white,No,No,Yes,Social Drinker,"Current Reformed Smoker, Duration Not Specified",yes,Melanoma,1,486
1,TCGA-HZ-8315,54,United States,not hispanic or latino,female,white,No,No,NaN,NaN,NaN,NaN,NaN,1,299
2,TCGA-HZ-8636,58,United States,not hispanic or latino,female,white,No,No,NaN,NaN,NaN,NaN,NaN,1,545
3,TCGA-IB-7888,66,Canada,not hispanic or latino,female,black or african american,No,No,NaN,NaN,NaN,NaN,NaN,1,1332
4,TCGA-IB-7893,64,Canada,not hispanic or latino,male,white,Yes,No,Yes,Social Drinker,Current Reformed Smoker for > 15 yrs,yes,Cancer,1,117
5,TCGA-US-A774,76,Australia,not hispanic or latino,female,white,No,No,Yes,Social Drinker,Current Reformed Smoker for > 15 yrs,yes,Cancer,1,695
6,TCGA-IB-AAUP,68,Canada,not hispanic or latino,male,white,Yes,No,No,Non-Drinker,Current Reformed Smoker for > 15 yrs,yes,Cancer,1,431
7,TCGA-IB-8127,59,Canada,not hispanic or latino,male,white,No,No,NaN,NaN,Current Reformed Smoker for < or = 15 yrs,no,NaN,1,522
8,TCGA-2J-AABU,56,United States,NaN,male,white,No,No,Yes,Heavy Drinker,Lifelong Non-Smoker,no,NaN,1,277
9,TCGA-HV-AA8V,50,South Korea,NaN,male,asian,No,No,Yes,Drinker,Current Reformed Smoker for < or = 15 yrs,NaN,NaN,1,920


### 6.2 Preprocesado de diagnosis

<p align="justify">En segundo lugar, se analizan los datos diagnósticos. Se seleccionan las columnas relevantes, se eliminan filas duplicadas y se uniformiza el nombre de las columnas. El DataFrame resultante se volverá a dividir en función de la clasificación del tumor, ya que, en función de esto hay una información disponible u otra.

In [27]:
# Selección de columnas.
diagnosis = clinical[['cases.submitter_id', 'diagnoses.ajcc_pathologic_stage',
                      'diagnoses.ajcc_pathologic_t', 'diagnoses.ajcc_pathologic_n',
                      'diagnoses.ajcc_pathologic_m', 'diagnoses.classification_of_tumor',
                      'diagnoses.primary_diagnosis', 'diagnoses.residual_disease',
                      'diagnoses.sites_of_involvement', 'diagnoses.tumor_grade',
                      'diagnoses.tissue_or_organ_of_origin',
                      'diagnoses.year_of_diagnosis']].copy()

# Eliminación de duplicados
diagnosis.drop_duplicates(inplace=True)
diagnosis.reset_index(drop=True, inplace=True)

# Formateo de las columnas.
diagnosis.columns = diagnosis.columns.str.replace('cases.', '')\
            .str.replace('diagnoses.', '').str.replace('submitter', 'patient')\
            .str.replace('_', ' ').str.title()

print(diagnosis.shape)
diagnosis

(307, 12)


,Patient Id,Ajcc Pathologic Stage,Ajcc Pathologic T,Ajcc Pathologic N,Ajcc Pathologic M,Classification Of Tumor,Primary Diagnosis,Residual Disease,Sites Of Involvement,Tumor Grade,Tissue Or Organ Of Origin,Year Of Diagnosis
0,TCGA-IB-7897,Stage IIB,T3,N1,M0,primary,"Infiltrating duct carcinoma, NOS",R0,Pancreas Head,G2,"Pancreas, NOS",2008
1,TCGA-IB-7897,NaN,NaN,NaN,NaN,metastasis,"Infiltrating duct carcinoma, NOS",NaN,NaN,NaN,Liver,NaN
2,TCGA-IB-7897,NaN,NaN,NaN,NaN,recurrence,"Infiltrating duct carcinoma, NOS",NaN,NaN,NaN,Liver,NaN
3,TCGA-HZ-8315,Stage IIA,T3,N0,MX,primary,"Infiltrating duct carcinoma, NOS",R1,Pancreas Head,G2,Head of pancreas,2012
4,TCGA-HZ-8636,Stage IV,T3,N0,M1,primary,"Infiltrating duct carcinoma, NOS",R1,Pancreas Tail,G3,Tail of pancreas,2012
5,TCGA-IB-7888,Stage IIA,T3,N0,M0,primary,"Infiltrating duct carcinoma, NOS",R0,Pancreas Head,G2,"Pancreas, NOS",2008
6,TCGA-IB-7888,NaN,NaN,NaN,NaN,metastasis,"Infiltrating duct carcinoma, NOS",NaN,NaN,NaN,Retroperitoneum,NaN
7,TCGA-IB-7893,Stage IIA,T3,N0,M0,primary,"Infiltrating duct carcinoma, NOS",R0,Pancreas Body,G3,"Pancreas, NOS",2009
8,TCGA-IB-7893,NaN,NaN,NaN,NaN,metastasis,"Infiltrating duct carcinoma, NOS",NaN,NaN,NaN,"Peritoneum, NOS",NaN
9,TCGA-US-A774,Stage IIB,T3,N1,MX,primary,"Infiltrating duct carcinoma, NOS",R0,Pancreas Head,G3,Head of pancreas,2011


#### 6.2.1: Preprocesado de diagnosis_prior

<p align="justify">Se seleccionan aquellas filas donde el tumor ha sido clasificado como previo al primario. Se eliminan las columnas vacías y aquellas que no aportan información. Se unifican el nombre de las categorías y se guarda. El DataFrame resultante contiene las siguientes columnas:

- Patient Id: Identificador.
- Primary Diagnosis: Diagnóstico primario.
- Tissue Or Organ Of Origin: Lugar donde se encuentra el cáncer.

In [28]:
# Selección de filas.
diagnosis_prior = diagnosis[diagnosis['Classification Of Tumor'] == 'Prior primary'].copy()

# Eliminación de columnas innecesarias.
diagnosis_prior = diagnosis_prior[['Patient Id', 'Primary Diagnosis',
                                   'Tissue Or Organ Of Origin']]
diagnosis_prior.reset_index(drop=True, inplace=True)

# Unificación de categorías.
diagnosis_prior['Primary Diagnosis'] = diagnosis_prior['Primary Diagnosis']\
        .str.replace('NOS', '').str.replace('in situ', '').str.title()\
        .str.replace('Malignant', '').str.replace(',', '').str.strip()

diagnosis_prior['Tissue Or Organ Of Origin'] = diagnosis_prior['Tissue Or Organ Of Origin']\
        .str.replace(', NOS', '').str.strip().str.title()

# Guardado.
save_data(diagnosis_prior, f"{output_path}diagnosis_prior.csv")

print(diagnosis_prior.shape)
diagnosis_prior

(23, 3)


,Patient Id,Primary Diagnosis,Tissue Or Organ Of Origin
0,TCGA-HZ-7289,Carcinoma,Skin
1,TCGA-F2-A44G,Infiltrating Duct Carcinoma,Breast
2,TCGA-3A-A9IL,Adenoma,Pituitary Gland
3,TCGA-3A-A9IL,Melanoma,Lower Limb
4,TCGA-IB-A5SS,Basal Cell Carcinoma,Skin
5,TCGA-IB-A5SS,Squamous Cell Carcinoma,Cervix Uteri
6,TCGA-2J-AABT,Intraductal Carcinoma Noninfiltrating,Breast
7,TCGA-FZ-5924,Melanoma,Skin
8,TCGA-2J-AABR,Basal Cell Carcinoma,Skin
9,TCGA-HZ-A4BK,Clear Cell Carcinoma,Kidney


#### 6.2.2: Preprocesado de diagnosis_primary

<p align="justify">Se seleccionan aquellas filas donde el tumor ha sido clasificado como primario. Se eliminan las columnas que no aportan información y se uniformiza las categorías. Se añade información de path_details. Se actualiza la información relacionada con la estadificación del tumor a la octava versión utilizando los datos añadidos de path_details cuando sea posible aplicando las funciones previamente definidas. Se guarda. El DataFrame resultante contiene las siguientes columnas:

- Patient Id: Identificador.
- Ajcc Pathologic Stage: Clasificación del estadio del cáncer.
- Ajcc Pathologic T: Clasificación dentro del sistema TNM.
- Ajcc Pathologic N: Clasificación dentro del sistema TNM.
- Ajcc Pathologic M: Clasificación dentro del sistema TNM.
- Primary Diagnosis: Diagnóstico primario.
- Residual Disease: Clasificación de la enfermedad residual.
- Sites Of Involvement: Zona del páncreas involucrada.
- Tumor Grade: Clasificación del grado del tumor.
- Year Of Diagnosis: Año del diagnóstico.
- Lymph Nodes Positive: Número de nódulos limfáticos afectador por el tumor.
- Maximum Tumor Diameter (cm): Diámetro máximo del tumor en centímetros.
- Ajcc Pathologic N 8th: Clasificación dentro del sistema TNM octava edición.
- Ajcc Pathologic T 8th: Clasificación dentro del sistema TNM octava edición.
- Ajcc Pathologic Stage 8th: Clasificación del estadio del cáncer octava edición.

In [29]:
def n_classificator_8th(row):
    nodes = row['Lymph Nodes Positive']

    if pd.isna(nodes):
        return row['Ajcc Pathologic N']

    if nodes == 0:
        return 'N0'
    elif nodes <= 3:
        return 'N1'
    elif nodes  > 3:
        return 'N2'
    else:
        return row['Ajcc Pathologic N']

def t_classificator_8th(row):
    if row['Ajcc Pathologic T'] == 'T4':
        return 'T4'

    diameter = row['Maximum Tumor Diameter (cm)']

    if pd.isna(diameter):
        return row['Ajcc Pathologic T']

    if diameter <= 2:
        return 'T1'
    elif diameter <= 4:
        return 'T2'
    elif diameter > 4:
        return 'T3'
    else:
        return row['Ajcc Pathologic T']

def stage_classificator_8th(row):
    if row['Ajcc Pathologic M'] == 'M1':
        return 'Stage IV'
    elif row['Ajcc Pathologic T 8th'] == 'T4' or row['Ajcc Pathologic N 8th'] == 'N2':
        return 'Stage III'
    elif row['Ajcc Pathologic T 8th'] == 'T3' or row['Ajcc Pathologic N 8th'] == 'N1':
        return 'Stage II'
    else:
        return 'Stage I'

In [30]:
# Selección de filas.
diagnosis_primary = diagnosis[diagnosis['Classification Of Tumor'] == 'primary'].copy()

# Eliminación de columnas innecesarias.
diagnosis_primary.drop(columns=['Classification Of Tumor',
                                'Tissue Or Organ Of Origin'], inplace=True)
diagnosis_primary.reset_index(drop=True, inplace=True)

# Unificación de categorías.
diagnosis_primary['Primary Diagnosis'] = diagnosis_primary['Primary Diagnosis']\
        .str.replace(', NOS', '').str.title()
stage_dict = {'Stage IA': 'Stage I', 'Stage IB': 'Stage I',
              'Stage IIA': 'Stage II','Stage IIB': 'Stage II',
              'Stage III': 'Stage III', 'Stage IV': 'Stage IV'}
diagnosis_primary['Ajcc Pathologic Stage'] = diagnosis_primary['Ajcc Pathologic Stage']\
        .map(stage_dict)

# Adición de columnas de path_details.
diagnosis_primary = diagnosis_primary.merge(path_details, on='Patient Id', how='left')

diagnosis_primary['Lymph Nodes Positive'] = diagnosis_primary['Lymph Nodes Positive']\
        .astype(float)
diagnosis_primary['Maximum Tumor Diameter (cm)'] = diagnosis_primary['Maximum Tumor Diameter (cm)']\
        .astype(float)

# Actualización estadificación tumor según última versión.
diagnosis_primary['Ajcc Pathologic N 8th'] = diagnosis_primary.apply(n_classificator_8th, axis=1)
diagnosis_primary['Ajcc Pathologic T 8th'] = diagnosis_primary.apply(t_classificator_8th, axis=1)
diagnosis_primary['Ajcc Pathologic Stage 8th'] = diagnosis_primary.apply(stage_classificator_8th, axis=1)

# Guardado.
save_data(diagnosis_primary, f"{output_path}diagnosis_primary.csv")

print(diagnosis_primary.shape)
diagnosis_primary

(185, 15)


,Patient Id,Ajcc Pathologic Stage,Ajcc Pathologic T,Ajcc Pathologic N,Ajcc Pathologic M,Primary Diagnosis,Residual Disease,Sites Of Involvement,Tumor Grade,Year Of Diagnosis,Lymph Nodes Positive,Maximum Tumor Diameter (cm),Ajcc Pathologic N 8th,Ajcc Pathologic T 8th,Ajcc Pathologic Stage 8th
0,TCGA-IB-7897,Stage II,T3,N1,M0,Infiltrating Duct Carcinoma,R0,Pancreas Head,G2,2008,2.0,3.5,N1,T2,Stage II
1,TCGA-HZ-8315,Stage II,T3,N0,MX,Infiltrating Duct Carcinoma,R1,Pancreas Head,G2,2012,0.0,6.0,N0,T3,Stage II
2,TCGA-HZ-8636,Stage IV,T3,N0,M1,Infiltrating Duct Carcinoma,R1,Pancreas Tail,G3,2012,0.0,9.0,N0,T3,Stage IV
3,TCGA-IB-7888,Stage II,T3,N0,M0,Infiltrating Duct Carcinoma,R0,Pancreas Head,G2,2008,0.0,3.0,N0,T2,Stage I
4,TCGA-IB-7893,Stage II,T3,N0,M0,Infiltrating Duct Carcinoma,R0,Pancreas Body,G3,2009,0.0,7.0,N0,T3,Stage II
5,TCGA-US-A774,Stage II,T3,N1,MX,Infiltrating Duct Carcinoma,R0,Pancreas Head,G3,2011,2.0,4.0,N1,T2,Stage II
6,TCGA-IB-AAUP,Stage II,T3,N1,M0,Infiltrating Duct Carcinoma,R1,Pancreas Head,G2,2013,7.0,2.7,N2,T2,Stage III
7,TCGA-IB-8127,Stage II,T3,N1,M0,Infiltrating Duct Carcinoma,R1,Pancreas Head,G2,2011,3.0,4.5,N1,T3,Stage II
8,TCGA-2J-AABU,Stage II,T3,N1,M0,Infiltrating Duct Carcinoma,R0,Pancreas Head,G3,2012,2.0,3.3,N1,T2,Stage II
9,TCGA-HV-AA8V,Stage II,T3,N1,MX,Infiltrating Duct Carcinoma,R2,Pancreas Head,G3,2012,3.0,NaN,N1,T3,Stage II


#### 6.2.3: Preprocesado de diagnosis_other

<p align="justify">Se seleccionan aquellas filas donde el tumor ha sido clasificado como recurrente, metastático o subsiguiente al primario. Se eliminan las columnas que no aportan información, se uniformiza las categorías y se guarda. El DataFrame resultante contiene las siguientes columnas:

- Patient Id: Identificador.
- Diagnosis Type: Tipo de diagnóstico.
- Tissue Or Organ Of Origin: Lugar donde se ha producido la recurrencia.

In [32]:
# Selección de filas.
diag_other = ['recurrence', 'metastasis', 'Subsequent Primary']
diagnosis_other = diagnosis[diagnosis['Classification Of Tumor'].isin(diag_other)].copy()

# Eliminación columnas innecesarias.
diagnosis_other = diagnosis_other[['Patient Id', 'Classification Of Tumor',
                                   'Tissue Or Organ Of Origin']]
diagnosis_other.reset_index(drop=True, inplace=True)
diagnosis_other.columns = ['Patient Id', 'Diagnosis Type', 'Tissue Or Organ Of Origin']

# Unificación de categorías.
diagnosis_other['Tissue Or Organ Of Origin'] = diagnosis_other['Tissue Or Organ Of Origin']\
        .str.replace(', NOS', '').str.title().str.replace('Specified Parts Of ', '')\
        .str.replace('Connective, Subcutaneous And Other Soft Tissues Of ', '')

# Eliminación fila duplicada.
diagnosis_other.drop(80, inplace=True)
diagnosis_other.reset_index(drop=True, inplace=True)

# Guardado.
save_data(diagnosis_other, f"{output_path}diagnosis_other.csv")

print(diagnosis_other.shape)
diagnosis_other

(98, 3)


,Patient Id,Diagnosis Type,Tissue Or Organ Of Origin
0,TCGA-IB-7897,metastasis,Liver
1,TCGA-IB-7897,recurrence,Liver
2,TCGA-IB-7888,metastasis,Retroperitoneum
3,TCGA-IB-7893,metastasis,Peritoneum
4,TCGA-US-A774,metastasis,Liver
5,TCGA-IB-8127,recurrence,Pancreas
6,TCGA-IB-8127,metastasis,Liver
7,TCGA-2J-AABU,metastasis,Liver
8,TCGA-HV-AA8V,metastasis,Bone
9,TCGA-3A-A9IH,metastasis,Lung


#### 6.2.4: Preprocesado de diagnosis_type

<p align="justify">Se genera una nueva tabla que recoge los diferentes diagnósticos que recibió cada paciente y se guarda. Se utilizará en posteriores análisis. El DataFrame resultante contiene las siguientes columnas:

- Patient Id: Identificador.
- Prior Primary: Existencia diagnóstico de primario previo.
- Primary: Existencia diagnóstico de primario.
- Recurrence: Existencia diagnóstico de recurrencia.
- Metastasis: Existencia diagnóstico de metastasis.
- Subsequent Primary: Existencia diagnóstico de nuevo primario.
- Total: Total de diagnósticos que recibió cada paciente.

In [33]:
# Obtención Patient Ids.
diagnosis_type = diagnosis_primary[['Patient Id']].copy()
recurrence = diagnosis_other[diagnosis_other['Diagnosis Type'] == 'recurrence']['Patient Id']
metastasis = diagnosis_other[diagnosis_other['Diagnosis Type'] == 'metastasis']['Patient Id']
subsequent = diagnosis_other[diagnosis_other['Diagnosis Type'] == 'Subsequent Primary']['Patient Id']

# Comprobación de diagnósticos.
diagnosis_type['Prior Primary'] = np.where(diagnosis_type['Patient Id']\
                                           .isin(diagnosis_prior['Patient Id']), 'Yes', 'No')
diagnosis_type['Primary'] = np.where(diagnosis_type['Patient Id']\
                                     .isin(diagnosis_primary['Patient Id']), 'Yes', 'No')
diagnosis_type['Recurrence'] = np.where(diagnosis_type['Patient Id']\
                                        .isin(recurrence), 'Yes', 'No')
diagnosis_type['Metastasis'] = np.where(diagnosis_type['Patient Id']\
                                        .isin(metastasis), 'Yes', 'No')
diagnosis_type['Subsequent Primary'] = np.where(diagnosis_type['Patient Id']\
                                                .isin(subsequent), 'Yes', 'No')
diagnosis_type['Total'] = diagnosis_type.apply(lambda row: (row != 'No').sum() - 1, axis=1)

# Guardado.
save_data(diagnosis_type, f"{output_path}diagnosis_type.csv")

print(diagnosis_type.shape)
diagnosis_type

(185, 7)


,Patient Id,Prior Primary,Primary,Recurrence,Metastasis,Subsequent Primary,Total
0,TCGA-IB-7897,No,Yes,Yes,Yes,No,3
1,TCGA-HZ-8315,No,Yes,No,No,No,1
2,TCGA-HZ-8636,No,Yes,No,No,No,1
3,TCGA-IB-7888,No,Yes,No,Yes,No,2
4,TCGA-IB-7893,No,Yes,No,Yes,No,2
5,TCGA-US-A774,No,Yes,No,Yes,No,2
6,TCGA-IB-AAUP,No,Yes,No,No,No,1
7,TCGA-IB-8127,No,Yes,Yes,Yes,No,3
8,TCGA-2J-AABU,No,Yes,No,Yes,No,2
9,TCGA-HV-AA8V,No,Yes,No,Yes,No,2


### 6.3: Preprocesado de treatment

<p align="justify">En tercer lugar, se analizan los datos relacionados con los tratamientos. Se seleccionan las columnas relevantes, se eliminan filas vacías y se uniformiza el nombre de las columnas. El DataFrame resultante se volverá a dividir en función del tipo de tratamiento, ya que, en función de esto hay una información disponible u otra.

In [65]:
# Selección de columnas.
treatment = clinical[['cases.submitter_id', 'treatments.days_to_treatment_end',
                      'treatments.days_to_treatment_start', 'treatments.initial_disease_status',
                      'treatments.number_of_fractions', 'treatments.residual_disease',
                      'treatments.submitter_id', 'treatments.therapeutic_agents',
                      'treatments.treatment_anatomic_sites', 'treatments.treatment_dose',
                      'treatments.treatment_dose_units', 'treatments.treatment_intent_type',
                      'treatments.treatment_or_therapy', 'treatments.treatment_outcome',
                      'treatments.treatment_type']].copy()

# Eliminación filas vacías.
treatment.dropna(thresh=2, inplace=True)
treatment.reset_index(drop=True, inplace=True)

# Formateo de las columnas.
treatment.columns = treatment.columns.str.replace('cases.submitter_id', 'patient_id')\
      .str.replace('treatments.', '').str.replace('_', ' ').str.title()

print(treatment.shape)
treatment

(1081, 15)


,Patient Id,Days To Treatment End,Days To Treatment Start,Initial Disease Status,Number Of Fractions,Residual Disease,Submitter Id,Therapeutic Agents,Treatment Anatomic Sites,Treatment Dose,Treatment Dose Units,Treatment Intent Type,Treatment Or Therapy,Treatment Outcome,Treatment Type
0,TCGA-IB-7897,NaN,NaN,NaN,NaN,NaN,TCGA-IB-7897_treatment2,NaN,NaN,NaN,NaN,Adjuvant,no,NaN,"Radiation Therapy, NOS"
1,TCGA-IB-7897,NaN,NaN,NaN,NaN,NaN,TCGA-IB-7897_treatment,NaN,NaN,NaN,NaN,Adjuvant,no,NaN,"Pharmaceutical Therapy, NOS"
2,TCGA-IB-7897,NaN,NaN,NaN,NaN,NaN,TCGA-IB-7897_treatment3,NaN,NaN,NaN,NaN,NaN,yes,NaN,Whipple
3,TCGA-IB-7897,NaN,NaN,Progressive Disease,NaN,NaN,TCGA-IB-7897_treatment8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Radiation Therapy, NOS"
4,TCGA-IB-7897,NaN,NaN,Progressive Disease,NaN,NaN,TCGA-IB-7897_treatment7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Pharmaceutical Therapy, NOS"
5,TCGA-IB-7897,NaN,NaN,Progressive Disease,NaN,NaN,TCGA-IB-7897_treatment9,NaN,NaN,NaN,NaN,NaN,no,NaN,"Surgery, NOS"
6,TCGA-IB-7897,NaN,NaN,Recurrent Disease,NaN,NaN,TCGA-IB-7897_treatment4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Pharmaceutical Therapy, NOS"
7,TCGA-IB-7897,NaN,NaN,Recurrent Disease,NaN,NaN,TCGA-IB-7897_treatment6,NaN,NaN,NaN,NaN,NaN,no,NaN,"Surgery, NOS"
8,TCGA-IB-7897,NaN,NaN,Recurrent Disease,NaN,NaN,TCGA-IB-7897_treatment5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Radiation Therapy, NOS"
9,TCGA-HZ-8315,NaN,NaN,NaN,NaN,NaN,TCGA-HZ-8315_treatment3,NaN,NaN,NaN,NaN,NaN,yes,NaN,Whipple


#### 6.3.1: Preprocesado de treatment_ebrt

<p align="justify">Se seleccionan aquellas filas donde el tipo de tratamiento es radioterapia de haz externo. Se modifica manualmente los valores de una fila, ya que, la información se encuentra dividida en dos filas mediante agregación para mantener solo una de esas filas. Se uniformizan las unidades de los datos para que toda la información este en las mismas unidades. Se eliminan las columnas vacías y aquellas que no aportan información, se adecua el nombre de las columnas y se guarda. El DataFrame resultante contiene las siguientes columnas:

- Patient Id: Identificador.
- Treatment Duration: Duración del tratamiento.
- Number Of Fractions: Número de fracciones.
- Treatment Anatomic Sites: Lugar del tratamiento.
- Treatment Dose (cGy): Dosis total recibida en centigray.
- Outcome: Resultado del tratamiento.

In [35]:
# Selección de filas.
treatment_ebrt = treatment[treatment['Treatment Type'] == 'Radiation, External Beam'].copy()

# Agregación manual.
treatment_ebrt.iloc[34, 4] = 28.
treatment_ebrt.iloc[34, 9] = 50
treatment_ebrt.drop_duplicates(subset=['Patient Id'], inplace=True, keep='last')

# Conversión de unidades.
treatment_ebrt['Treatment Dose'] = treatment_ebrt['Treatment Dose'].astype(float)
condition = treatment_ebrt['Treatment Dose Units'] == 'Gy'
treatment_ebrt.loc[condition, 'Treatment Dose'] = treatment_ebrt.loc[condition, 'Treatment Dose'] * 100

# Eliminación columnas vacías e innecesarias.
treatment_ebrt.dropna(how='all', inplace=True, axis=1)
treatment_ebrt.drop(columns=['Submitter Id', 'Treatment Or Therapy',
                             'Treatment Type', 'Days To Treatment Start',
                             'Treatment Dose Units'], inplace=True)
treatment_ebrt.reset_index(drop=True, inplace=True)

# Adecuación nombre de las columnas.
treatment_ebrt.columns = ['Patient Id', 'Treatment Duration',
                          'Number Of Fractions', 'Treatment Anatomic Sites',
                          'Treatment Dose (cGy)', 'Outcome']

# Guardado.
save_data(treatment_ebrt, f"{output_path}treatment_ebrt.csv")

print(treatment_ebrt.shape)
treatment_ebrt

(41, 6)


,Patient Id,Treatment Duration,Number Of Fractions,Treatment Anatomic Sites,Treatment Dose (cGy),Outcome
0,TCGA-FB-AAPP,166,25.0,Primary Tumor Field,4500.0,Progressive Disease
1,TCGA-HZ-7289,467,NaN,Distant Site,4500.0,Progressive Disease
2,TCGA-HZ-7920,120,28.0,Primary Tumor Field,5040.0,Complete Response
3,TCGA-F2-A44G,144,28.0,Primary Tumor Field,5440.0,Complete Response
4,TCGA-HZ-A49I,93,25.0,Primary Tumor Field,5040.0,Progressive Disease
5,TCGA-2J-AABI,83,28.0,Primary Tumor Field,5040.0,NaN
6,TCGA-LB-A9Q5,238,28.0,Primary Tumor Field,5000.0,NaN
7,TCGA-FZ-5921,112,5.0,Primary Tumor Field,2000.0,Progressive Disease
8,TCGA-3A-A9I9,134,25.0,Primary Tumor Field,NaN,Stable Disease
9,TCGA-HZ-8637,243,28.0,Primary Tumor Field,5040.0,Complete Response


#### 6.3.2: Preprocesado de treatment_chemo

<p align="justify">Se seleccionan aquellas filas donde el tipo de tratamiento es quimioterapia o tratamiento auxiliar, ya que, se considera que ambos tratamientos se utilizaron simultáneamente.
<p align="justify">Se eliminan las columnas vacías y aquellas que no aportan información, se adecua el nombre de las columnas. Se uniformizan las categorías y se agrupan las filas que tienen el mismo paciente, duración y resultado. Si cumplen esta condición se considera que los fármacos forman parte del mismo tratamiento.
<p align="justify">A continuación, se aplican las siguientes suposiciones para consolidar las diferentes categorías de Agents:

- No se hacen distinciones entre diferentes formas del mismo agente. Se eliminan las terminaciones 'Hydrochloride' y 'Calcium'.
- Se supone que Fluorouracil siempre se suministra junto con Leucovorin.
- Se elimina una fila que se considera duplicada.
- Se corrige manualmente un valor faltante que se puede inferir a partir de los otros datos.
- Si hay filas con el mismo paciente, tratamiento y resultado se consideran duplicadas y se conserva la más reciente.
- Se combina los valores de dos filas manualmente en dos ocasiones.
- Si hay filas con el mismo Patient Id y Outcome catalogadas como monotherapy se consideran el mismo tratamiento y se agrupan.

Finalmente, se guarda. El DataFrame resultante contiene las siguientes columnas:

- Patient Id: Identificador.
- Treatment Duration: Duración del tratamiento.
- Outcome: Resultado del tratamiento.
- Agents: Fármacos utilizados.
- Treatment Type: Tipo de tratamiento, monoterapia o combinación.

In [66]:
def treatment_type(row):
    monotherapy = ['Fluorouracil Leucovorin', 'Gemcitabine', 'Capecitabine', 'Cisplatin',
               'Nab-paclitaxel', 'Oxaliplatin', 'Irinotecan', 'Erlotinib', 'Fluorouracil']

    if row['Agents'] is np.nan:
        return np.nan
    elif row['Agents'] in monotherapy:
        return 'Monotherapy'
    else:
        return 'Combination'

In [87]:
# Selección filas.
treatment_chemo = treatment[treatment['Treatment Type']\
                            .isin(['Chemotherapy', 'Ancillary Treatment'])].copy()

# Eliminación columnas vacías e innecesarias.
treatment_chemo.dropna(how='all', inplace=True, axis=1)
treatment_chemo.drop(columns=['Submitter Id', 'Treatment Or Therapy',
                              'Treatment Type', 'Days To Treatment Start'],
                     inplace=True)

# Adecuación nombres columnas.
treatment_chemo.columns = ['Patient Id', 'Treatment Duration',
                           'Agents', 'Outcome']

# Formateo nombres categorías.
treatment_chemo.replace(np.nan, '0', inplace=True)
treatment_chemo['Agents'] = treatment_chemo['Agents'].str.replace(r'Hydrochloride|Calcium', '', regex=True)\
              .str.replace('Leucovorin', '')

# Agrupación de mismos tratamientos.
treatment_chemo = treatment_chemo.groupby(['Patient Id', 'Treatment Duration',
                                           'Outcome'], as_index=False).sum()

# Formateo nombres categorías.
treatment_chemo['Agents'] = treatment_chemo['Agents'].str.split(r'(?=[A-Z])')\
            .str.join(' ').str.replace(',', '').str.strip()
treatment_chemo['Agents'] = [' '.join(sorted(x)) for x in treatment_chemo['Agents'].str.split()]
treatment_chemo.replace('0', np.nan, inplace=True)

# Creación nuevas columnas.
treatment_chemo['Treatment Type'] = treatment_chemo.apply(treatment_type, axis=1)

# Eliminación fila duplicada.
treatment_chemo.drop(124, inplace=True)
treatment_chemo.reset_index(drop=True, inplace=True)

# Corrección manual valor faltante.
treatment_chemo.iloc[159, 3] = 'Fluorouracil Leucovorin'

# Eliminación filas duplicadas.
treatment_chemo.drop_duplicates(subset=['Patient Id', 'Outcome', 'Agents'], inplace=True, keep='last')
treatment_chemo.reset_index(drop=True, inplace=True)

# Corrección manual.
treatment_chemo.iloc[24, 3] = 'Fluorouracil Leucovorin Gemcitabine'
treatment_chemo.iloc[24, 4] = 'Combination'
treatment_chemo.drop(26 ,inplace=True)
treatment_chemo.reset_index(drop=True, inplace=True)

treatment_chemo.iloc[45, 3] = 'Capecitabine Docetaxel Gemcitabine'
treatment_chemo.drop(44, inplace=True)

# Agrupación mismos tratamientos.
chemo_combo = treatment_chemo[treatment_chemo['Treatment Type'] == 'Combination'].copy()
chemo_combo.drop(columns=['Treatment Type'], inplace=True)
chemo_mono = treatment_chemo[treatment_chemo['Treatment Type'] == 'Monotherapy'].copy()
chemo_mono.drop(columns=['Treatment Type'], inplace=True)

chemo_mono = chemo_mono.groupby(['Patient Id', 'Outcome'], as_index=False).agg({'Treatment Duration': 'max',
                                                                              'Agents': 'sum'})
# Formateo nombres categorías.
chemo_mono['Agents'] = chemo_mono['Agents'].str.split(r'(?=[A-Z])')\
            .str.join(' ').str.replace(',', '').str.strip()
chemo_mono['Agents'] = [' '.join(sorted(x)) for x in chemo_mono['Agents'].str.split()]

# Unión de DataFrames.
treatment_chemo = pd.concat([chemo_mono, chemo_combo])

# Formateo categorías Agents.
treatment_chemo['Agents'] = treatment_chemo['Agents'].str.replace('Fluorouracil', 'Fluorouracil Leucovorin')\
            .str.replace('Fluorouracil Leucovorin Irinotecan Oxaliplatin', 'FOLFIRINOX')\
            .str.replace('Fluorouracil Leucovorin Oxaliplatin', 'FOLFOX/XELOX')\
            .str.replace('Fluorouracil Leucovorin Irinotecan', 'FOLFIRI')\
            .str.replace('Fluorouracil Leucovorin Leucovorin Gemcitabine', 'Fluorouracil Leucovorin Gemcitabine')\
            .str.replace('Capecitabine Oxaliplatin', 'FOLFOX/XELOX')

# Creación nuevas columnas.
treatment_chemo['Treatment Type'] = treatment_chemo.apply(treatment_type, axis=1)
treatment_chemo.sort_values(by=['Patient Id'], inplace=True)
treatment_chemo.reset_index(drop=True, inplace=True)

# Guardado.
save_data(treatment_chemo, f"{output_path}treatment_chemo.csv")

print(treatment_chemo.shape)
treatment_chemo

(119, 5)


,Patient Id,Outcome,Treatment Duration,Agents,Treatment Type
0,TCGA-2L-AAQA,Progressive Disease,126,Gemcitabine,Monotherapy
1,TCGA-2L-AAQE,Progressive Disease,NaN,Gemcitabine,Monotherapy
2,TCGA-2L-AAQI,Progressive Disease,72,Gemcitabine,Monotherapy
3,TCGA-2L-AAQJ,Progressive Disease,171,Gemcitabine,Monotherapy
4,TCGA-3A-A9I5,Complete Response,211,Fluorouracil Leucovorin Gemcitabine,Combination
5,TCGA-3A-A9I7,NaN,139,Fluorouracil Leucovorin Gemcitabine,Combination
6,TCGA-3A-A9I9,Stable Disease,265,Gemcitabine,Monotherapy
7,TCGA-3A-A9I9,Progressive Disease,620,FOLFOX/XELOX,Combination
8,TCGA-3A-A9I9,Stable Disease,336,FOLFOX/XELOX,Combination
9,TCGA-3A-A9IC,Progressive Disease,722,Gemcitabine,Monotherapy


#### 6.3.3 Preprocesado de treatment_type

<p align="justify">Se genera una nueva tabla que recoge los diferentes tratamientos que recibió cada paciente y se guarda. Se utilizará en posteriores análisis. El DataFrame resultante contiene las siguientes columnas:

- Patient Id: Identificador.
- EBRT: Existencia de tratamiento de EBRT.
- Chemotherapy: Existencia de tratamiento de quimioterapia sin contar adyuvante.
- Adjuvant: Existencia de tratamiento adyuvante y tipo.
- Pancreatectomy: Existencia de tratamiento de pancreatectomía y tipo.
- Surgery: Existencia de tratamiento de cirugía.
- Pharmaceutical Therapy: Existencia de tratamiento farmacéutico sin contar adyuvante.
- Radiation Therapy: Existencia de tratamiento de radioterapia.
- Whipple: Existencia de tratamiento Whipple.
- Other: Existencia de tratamiento de otros tipos y su tipo.
- Total: Total de tratamientos que recibió cada paciente.

In [41]:
# Obtención Ids.
treatment_type = demographic[['Patient Id']].copy()

# Creación datos adjuvant.
adjuvant = treatment[treatment['Treatment Intent Type'] == 'Adjuvant'].copy()
adjuvant = adjuvant[['Patient Id', 'Treatment Intent Type', 'Treatment Type']]

adjuvant = adjuvant.groupby(['Patient Id', 'Treatment Intent Type'], as_index=False).sum()
adjuvant['Treatment Type'] = adjuvant['Treatment Type']\
          .str.replace(', NOS', ' ').str.strip().str.replace('Therapy', '')
adjuvant['Treatment Type'] = [' '.join(sorted(x)) for x in adjuvant['Treatment Type'].str.split()]

# Obtención Ids de cada tratamiento.
prancreatectomy = treatment[treatment['Treatment Type']\
                                     .isin(['Distal Pancreatectomy',
                                            'Total Pancreatectomy'])][['Patient Id',
                                                                       'Treatment Type']]
surgery = treatment[treatment['Treatment Type'] == 'Surgery, NOS']['Patient Id']
pharmaceutical = treatment[(treatment['Treatment Type'] == 'Pharmaceutical Therapy, NOS') &
                           (treatment['Treatment Intent Type'] != 'Adjuvant')]['Patient Id']
radiation = treatment[(treatment['Treatment Type'] == 'Radiation Therapy, NOS') &
                      (treatment['Treatment Intent Type'] != 'Adjuvant')]['Patient Id']
whipple = treatment[treatment['Treatment Type'] == 'Whipple']['Patient Id']

# Creación datos other treatments.
other_treatments = ['Hormone Therapy', 'Biopsy, Excisional', 'Immunotherapy (Including Vaccines)']
treatment_other = treatment[treatment['Treatment Type'].isin(other_treatments)].copy()
treatment_other.reset_index(drop=True, inplace=True)

treatment_other = treatment_other[['Patient Id', 'Therapeutic Agents', 'Treatment Type']]
treatment_other.drop(1, inplace=True)
treatment_other['Treatment Type'] = treatment_other['Treatment Type']\
      .str.replace('Hormone Therapy', 'Megestrol Acetate & Dexamethasone - Hormone Therapy')\
      .str.replace('Immunotherapy (Including Vaccines)', 'Antineoplastic Vaccine - Immunotherapy')

# Comprobación tratamientos.
treatment_type['EBRT'] = np.where(treatment_type['Patient Id']\
                                  .isin(treatment_ebrt['Patient Id']), 'Yes', 'No')
treatment_type['Chemotherapy'] = np.where(treatment_type['Patient Id']\
                                          .isin(treatment_chemo['Patient Id']), 'Yes', 'No')
treatment_type['Adjuvant'] = treatment_type['Patient Id'].map(
        adjuvant.set_index('Patient Id')['Treatment Type']).fillna('No')
treatment_type['Pancreatectomy'] = treatment_type['Patient Id'].map(
        prancreatectomy.set_index('Patient Id')['Treatment Type']).fillna('No')
treatment_type['Surgery'] = np.where(treatment_type['Patient Id'].isin(surgery), 'Yes', 'No')
treatment_type['Pharmaceutical Therapy'] = np.where(treatment_type['Patient Id']\
                                                    .isin(pharmaceutical), 'Yes', 'No')
treatment_type['Radiation Therapy'] = np.where(treatment_type['Patient Id']\
                                               .isin(radiation), 'Yes', 'No')
treatment_type['Whipple'] = np.where(treatment_type['Patient Id']\
                                     .isin(whipple), 'Yes', 'No')
treatment_type['Other'] = treatment_type['Patient Id'].map(
        treatment_other.set_index('Patient Id')['Treatment Type']).fillna('No')
treatment_type['Total'] = treatment_type.apply(lambda row: (row != 'No').sum() - 1, axis=1)

# Guardado.
save_data(treatment_type, f"{output_path}treatment_type.csv")

print(treatment_type.shape)
treatment_type

(185, 11)


,Patient Id,EBRT,Chemotherapy,Adjuvant,Pancreatectomy,Surgery,Pharmaceutical Therapy,Radiation Therapy,Whipple,Other,Total
0,TCGA-IB-7897,No,No,Pharmaceutical Radiation,No,Yes,Yes,Yes,Yes,No,5
1,TCGA-HZ-8315,No,No,Pharmaceutical Radiation,No,No,No,No,Yes,No,2
2,TCGA-HZ-8636,No,No,Pharmaceutical Radiation,No,Yes,No,No,No,No,2
3,TCGA-IB-7888,No,Yes,Radiation,No,Yes,Yes,Yes,Yes,No,6
4,TCGA-IB-7893,No,No,Pharmaceutical Radiation,Distal Pancreatectomy,Yes,Yes,Yes,No,No,5
5,TCGA-US-A774,No,Yes,Radiation,No,Yes,Yes,Yes,Yes,No,6
6,TCGA-IB-AAUP,No,Yes,Radiation,No,Yes,No,No,No,No,3
7,TCGA-IB-8127,No,Yes,Radiation,No,Yes,Yes,Yes,Yes,No,6
8,TCGA-2J-AABU,No,No,Radiation,No,Yes,Yes,Yes,Yes,No,5
9,TCGA-HV-AA8V,No,Yes,No,No,Yes,Yes,Yes,No,No,4


## 7. Preprocesado de mutation

In [83]:
url_archivo = "https://linkedomics.org/data_download/TCGA-PAAD/Human__TCGA_PAAD__WUSM__Mutation__GAIIx__01_28_2016__BI__Gene__Firehose_MutSig2CV.cbt"

nombre_archivo_local = f"{input_path}mutation_gene.cbt"

response = requests.get(url_archivo, stream=True)

with open(nombre_archivo_local, 'wb') as f:
    for chunk in response.iter_content(chunk_size=8192):
        f.write(chunk)

print(f"El archivo se guardó como: {nombre_archivo_local}")

El archivo se guardó como: data/raw/mutation_gene.cbt


In [86]:
mutation_gene = read_data(f"{input_path}mutation_gene.cbt", sep='\t')

mutation_gene.columns = mutation_gene.columns.str.replace('.', '-')
mutation_gene.loc[:, 'Total'] = mutation_gene.iloc[:, 1:].sum(axis=1)
mutation_gene.loc[:, 'Total %'] = mutation_gene['Total'] * 100 / 126
mutation_gene.sort_values(by=['Total'], ascending=False, inplace=True)
mutation_gene.reset_index(drop=True, inplace=True)
mutation_gene.rename(columns={'attrib_name': 'Gene'}, inplace=True)

# Guardado.
save_data(mutation_gene, f"{output_path}mutation_gene.csv")

print(mutation_gene.shape)
mutation_gene.head(20)

(2317, 129)


,Gene,TCGA-3A-A9IU,TCGA-IB-7651,TCGA-2L-AAQA,TCGA-2J-AABF,TCGA-H8-A6C1,TCGA-XN-A8T5,TCGA-LB-A8F3,TCGA-FZ-5919,TCGA-HV-A7OL,TCGA-2J-AAB4,TCGA-2L-AAQI,TCGA-FB-A78T,TCGA-HZ-8637,TCGA-HZ-A8P0,TCGA-RB-AA9M,TCGA-IB-AAUR,TCGA-Q3-AA2A,TCGA-FB-AAQ1,TCGA-IB-8126,TCGA-IB-AAUN,TCGA-US-A776,TCGA-YB-A89D,TCGA-2L-AAQJ,TCGA-FB-AAPU,TCGA-IB-A5SP,TCGA-RB-A7B8,TCGA-FB-AAQ0,TCGA-IB-A5SO,TCGA-HV-A7OP,TCGA-LB-A7SX,TCGA-LB-A9Q5,TCGA-YY-A8LH,TCGA-3A-A9IS,TCGA-XN-A8T3,TCGA-HZ-A8P1,TCGA-HV-AA8V,TCGA-2J-AAB1,TCGA-XD-AAUI,TCGA-2J-AAB9,TCGA-2J-AABH,TCGA-2J-AABP,TCGA-3A-A9I5,TCGA-FZ-5921,TCGA-HZ-A77P,TCGA-IB-A5ST,TCGA-IB-AAUU,TCGA-3A-A9J0,TCGA-2L-AAQE,TCGA-3A-A9I9,TCGA-2J-AABO,TCGA-FZ-5920,TCGA-FB-A5VM,TCGA-RL-AAAS,TCGA-FB-AAQ6,TCGA-HZ-A77Q,TCGA-FZ-5923,TCGA-2J-AABE,TCGA-3A-A9IJ,TCGA-3A-A9IZ,TCGA-3A-A9IH,TCGA-IB-7885,TCGA-FB-AAPZ,TCGA-YH-A8SY,TCGA-HZ-A9TJ,TCGA-US-A77J,TCGA-3A-A9IO,TCGA-HV-AA8X,TCGA-F2-A7TX,TCGA-2J-AABA,TCGA-3E-AAAY,TCGA-FZ-5922,TCGA-IB-A5SQ,TCGA-IB-A6UG,TCGA-OE-A75W,TCGA-3A-A9IB,TCGA-IB-A5SS,TCGA-US-A77G,TCGA-S4-A8RO,TCGA-IB-A7M4,TCGA-2J-AAB8,TCGA-2J-AABV,TCGA-XD-AAUG,TCGA-Z5-AAPL,TCGA-FB-AAPS,TCGA-2L-AAQM,TCGA-HZ-A77O,TCGA-FB-AAPP,TCGA-FB-AAQ2,TCGA-2L-AAQL,TCGA-IB-A7LX,TCGA-IB-AAUQ,TCGA-XD-AAUL,TCGA-2J-AABU,TCGA-US-A779,TCGA-2J-AABT,TCGA-IB-AAUS,TCGA-F2-7273,TCGA-FB-AAPQ,TCGA-IB-AAUO,TCGA-FZ-5926,TCGA-HZ-7925,TCGA-US-A77E,TCGA-IB-7652,TCGA-F2-A8YN,TCGA-3A-A9IC,TCGA-IB-AAUM,TCGA-HZ-7918,TCGA-Q3-A5QY,TCGA-IB-A6UF,TCGA-3A-A9IX,TCGA-IB-AAUW,TCGA-2J-AABR,TCGA-IB-AAUP,TCGA-3E-AAAZ,TCGA-XD-AAUH,TCGA-3A-A9IL,TCGA-2J-AABK,TCGA-3A-A9IV,TCGA-IB-AAUV,TCGA-2J-AAB6,TCGA-3A-A9I7,TCGA-HZ-A49I,TCGA-FB-AAQ3,TCGA-FZ-5924,TCGA-F2-A44G,TCGA-3A-A9IR,Total,Total %
0,KRAS,1,1,1,1,1,1,0,1,1,1,1,0,1,1,1,1,1,1,0,1,0,1,1,1,1,1,1,1,1,0,1,1,0,1,1,1,1,1,1,1,0,0,1,1,1,1,0,1,1,1,1,1,0,1,1,1,1,0,1,1,1,1,1,1,0,0,1,1,0,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,0,1,0,1,1,1,1,1,1,1,1,1,0,1,1,1,1,0,1,1,1,1,0,0,1,1,1,1,1,1,0,0,1,0,0,1,1,1,1,1,1,0,100,79.365079
1,TP53,1,0,1,1,1,1,0,1,1,1,1,1,1,0,1,0,0,1,0,1,0,0,1,0,1,0,1,1,0,1,1,0,0,1,0,1,1,1,1,1,0,0,1,1,1,1,1,1,1,1,1,1,0,1,1,0,0,0,1,1,0,1,1,1,0,0,1,1,0,0,0,0,1,1,1,1,0,1,1,1,1,1,1,1,0,1,0,1,1,0,1,1,1,1,1,1,0,1,0,1,1,0,1,1,1,0,0,0,1,1,0,1,1,0,1,0,1,0,0,1,0,1,1,1,0,0,81,64.285714
2,TTN,0,1,0,1,0,0,0,1,0,0,0,1,0,0,1,0,1,0,0,0,1,0,0,0,1,1,1,0,1,0,1,1,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1,0,1,0,1,0,1,1,0,1,0,0,0,0,1,1,0,0,1,1,1,0,0,1,0,1,0,1,0,0,0,1,0,0,0,1,0,0,1,0,1,0,0,1,0,0,1,0,0,0,0,1,1,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,1,44,34.920635
3,MAMLD1,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,1,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,1,0,1,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,1,0,1,0,0,0,0,0,0,28,22.222222
4,CDKN2A,1,0,0,1,0,0,0,0,1,0,0,1,0,1,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,1,1,1,1,1,0,1,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,27,21.428571
5,SMAD4,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,1,0,1,0,0,1,0,0,0,0,0,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,26,20.634921
6,FAM47C,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,1,0,1,1,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,21,16.666667
7,MAGEC1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,1,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,21,16.666667
8,TMC4,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,1,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,